目录

一、长宽表的变形

    1. pivot
    2. pivot_table
    练一练
    3. melt
    4. wide_to_long

二、索引的变形

    1. stack与unstack
    2. 聚合与变形的关系

三、其他变形函数

    1. crosstab
    2. explode
    3. get_dummies

四、练习

    Ex1：美国非法药物数据集
    Ex2：特殊的wide_to_long方法

<center><h1>第五章 变形</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、长宽表的变形

In [2]:
#一个表中把性别存储在某一个列中，那么它就是关于性别的长表
pd.DataFrame({'Gender':['F','F','M','M'], 'Height':[163, 160, 175, 180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
#如果把性别作为列名，列中的元素是某一其他的相关特征数值，那么这个表是关于性别的宽表
pd.DataFrame({'Height: F':[163, 160], 'Height: M':[175, 180]})

,Height: F,Height: M
0,163,175
1,160,180


这两张表的呈现方式不同，而其呈现方式主要又与性别一列选择的布局模式有关，即到底是以$\color{red}{long}$的状态存储还是以$\color{red}{wide}$的状态存储。因此，`pandas`针对此类长宽表的变形操作设计了一些有关的变形函数。

1. pivot

`pivot`是长表变宽表的函数

In [4]:
#下表存储了张三和李四的语文和数学分数，现在想要把语文和数学分数作为列来展示。
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [5]:
#长变宽操作:变形后的行索引、需要转到列索引的列，以及这些列和行索引对应的数值，它们分别对应了pivot方法中的index, columns, values参数
#新生成表的列索引是columns对应列的unique值，而新表的行索引是index对应列的unique值，而values对应了想要展示的数值列。
df.pivot(index='Name', columns='Subject', values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


通过颜色的标记，更容易地能够理解其变形的过程：

<img src="./source/_static/ch5_pivot.png" width="20%">

In [7]:
#利用pivot进行变形操作需要满足唯一性:新表中的行列索引对应了唯一的value,原表中的index和columns对应两个列的行组合必须唯一
#Name与Subject的组合中两次出现("San Zhang", "Chinese"),不能确定变形后是填写80分还是75分
df.loc[1, 'Subject'] = 'Chinese'
try:
    df.pivot(index='Name', columns='Subject', values='Grade')
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

In [8]:
#下表中六列分别为班级、姓名、测试类型（期中考试和期末考试）、科目、成绩、排名。
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                              'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                    'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                                 'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})
df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [9]:
#pivot相关的三个参数允许被设置为列表，这也意味着会返回多级索引。
#把测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引，并且同时统计成绩和排名
pivot_multi = df.pivot(index = ['Class', 'Name'],
                       columns = ['Subject','Examination'],
                       values = ['Grade','rank'])
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

根据唯一性原则，新表的行索引等价于对`index`中的多列使用`drop_duplicates`，而列索引的长度为`values`中的元素个数乘以`columns`的唯一组合数量（与`index`类似） 。从下面的示意图中能够比较容易地理解相应的操作：

<img src="./source/_static/ch5_mulpivot.png" width="35%">


2. pivot_table



In [10]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang', 
                              'San Zhang', 'San Zhang',
                              'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                                 'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [11]:
#pivot的使用依赖于唯一性条件，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值。
#pandas中提供了pivot_table来实现，其中的aggfunc参数就是使用的聚合函数。
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [12]:
#传入以序列为输入标量为输出的聚合函数来实现自定义操作,与上述功能等价
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x:x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [13]:
#pivot_table具有边际汇总的功能，可以通过设置margins=True来实现
#统计了语文均分和数学均分、张三均分和李四均分，以及总体所有分数的均分
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc='mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


【练一练】
在上面的边际汇总例子中，行或列的汇总为新表中行元素或者列元素的平均值，而总体的汇总为新表中四个元素的平均值。这种关系一定成立吗？若不成立，请给出一个例子来说明。

In [33]:
#一定成立

df1.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc='mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,84,75,79.5
Si Li,90,85,87.5
All,87,80,83.5


3、melt函数

In [34]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [35]:
#melt函数：把宽表转为长表
#将Subject压缩到一个列中。
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


`melt`的主要参数和压缩的过程如下图所示：

<img src="./source/_static/ch5_melt.png" width="35%">


In [36]:
#melt和pivot是一组互逆过程，可以通过pivot操作把df_melted转回df的形式
df_unmelted = df_melted.pivot(index = ['Class', 'Name'],
                              columns='Subject',
                              values='Grade')
df_unmelted # 下面需要恢复索引，并且重命名列索引名称

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,80
2,Si Li,90,75


In [37]:
df_unmelted = df_unmelted.reset_index().rename_axis(columns={'Subject':''})
df_unmelted.equals(df)

True

4. wide_to_long


In [39]:

df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang', 'Si Li'],
                   'Chinese_Mid':[80, 75], 'Math_Mid':[90, 85],
                   'Chinese_Final':[80, 75], 'Math_Final':[90, 85]})
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,85,75,85


In [40]:
#melt方法中，在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，即values_name。
#列中包含了交叉类别，想要把values_name对应的Grade扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩
pd.wide_to_long(df,
                stubnames=['Chinese', 'Math'],
                i = ['Class', 'Name'],
                j='Examination',
                sep='_',
                suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

具体的变换过程由下图进行展示，属相同概念的元素使用了一致的颜色标出：

<img src="./source/_static/ch5_wtl.png" width="35%">


In [42]:
#把之前在`pivot`一节中多列操作的结果（产生了多级索引），利用`wide_to_long`函数，将其转为原来的形态。

res = pivot_multi.copy()
res.columns = res.columns.map(lambda x:'_'.join(x))
res = res.reset_index()
res = pd.wide_to_long(res, stubnames=['Grade', 'rank'],
                           i = ['Class', 'Name'],
                           j = 'Subject_Examination',
                           sep = '_',
                           suffix = '.+')
res

Grade  rank
Class Name      Subject_Examination             
1     San Zhang Chinese_Mid             80    10
                Chinese_Final           75    15
                Math_Mid                90    20
                Math_Final              85     7
2     Si Li     Chinese_Mid             85    21
                Chinese_Final           65    15
                Math_Mid                92     6
                Math_Final              88     2

In [43]:
res = res.reset_index()
res[['Subject', 'Examination']] = res['Subject_Examination'].str.split('_', expand=True)#`str.split`函数，对序列按照某个分隔符进行拆分
res = res[['Class', 'Name', 'Examination', 'Subject', 'Grade', 'rank']].sort_values('Subject')
res = res.reset_index(drop=True)
res

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


## 二、索引的变形

### 1. stack与unstack

在第二章中提到了利用`swaplevel`或者`reorder_levels`进行索引内部的层交换，下面就要讨论$\color{red}{行列索引之间}$的交换，由于这种交换带来了`DataFrame`维度上的变化，因此属于变形操作。在第一节中提到的4种变形函数与其不同之处在于，它们都属于某一列或几列$\color{red}{元素}$和$\color{red}{列索引}$之间的转换，而不是索引之间的转换。


In [44]:
#行列索引之间 的交换，由于这种交换带来了DataFrame维度上的变化，因此属于变形操作。
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['col_1', 'col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [45]:
#unstack函数的作用是把行索引转为列索引
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [46]:
#unstack的主要参数是移动的层号，默认转化最内层，移动到列索引的最内层，同时支持同时转化多个层：
df.unstack(2)

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [47]:
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

In [48]:
#类似于pivot中的唯一性要求，在unstack中必须保证 被转为列索引的行索引层 和 被保留的行索引层 构成的组合是唯一的
#把前两个列索引改成相同的破坏唯一性，那么就会报错
my_index = df.index.to_list()
my_index[1] = my_index[0]
df.index = pd.Index(my_index)
df

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [49]:
try:
    df.unstack()
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

In [50]:
#与unstack相反，stack的作用就是把列索引的层压入行索引，其用法完全类似。
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['index_1', 'index_2']).T
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [51]:
df.stack()

A         B     
               cat  dog  cat  dog
index_1 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0
index_2 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0

In [52]:
df.stack([1, 2])

A    B
index_1 cat big    1.0  1.0
        dog small  1.0  1.0
index_2 cat big    1.0  1.0
        dog small  1.0  1.0

### 2. 聚合与变形的关系

在上面介绍的所有函数中，除了带有聚合效果的`pivot_table`以外，所有的函数在变形前后并不会带来`values`个数的改变，只是这些值在呈现的形式上发生了变化。在上一章讨论的分组聚合操作，由于生成了新的行列索引，因此必然也属于某种特殊的变形操作，但由于聚合之后把原来的多个值变为了一个值，因此`values`的个数产生了变化，这也是分组聚合与变形函数的最大区别。

## 三、其他变形函数

### 1. crosstab



In [54]:
df = pd.read_csv('./data/learn_pandas.csv')
pd.crosstab(index = df.School, columns = df.Transfer)
#在默认状态下，crosstab可以统计元素组合出现的频数，即count操作。例如统计learn_pandas数据集中学校和转系情况对应的频数

Transfer,N,Y
School,,
Fudan University,38,1
Peking University,28,2
Shanghai Jiao Tong University,53,0
Tsinghua University,62,4


In [55]:
#这等价于如下crosstab的如下写法，这里的aggfunc即聚合参数
pd.crosstab(index = df.School, columns = df.Transfer, values = [0]*df.shape[0], aggfunc = 'count')

Transfer,N,Y
School,,
Fudan University,38.0,1.0
Peking University,28.0,2.0
Shanghai Jiao Tong University,53.0,NaN
Tsinghua University,62.0,4.0


In [56]:
#可以利用pivot_table进行等价操作
#统计的是组合的频数，values参数无论传入哪一个列都不会影响最后的结果
df.pivot_table(index = 'School',
               columns = 'Transfer',
               values = 'Name',
               aggfunc = 'count')

Transfer,N,Y
School,,
Fudan University,38.0,1.0
Peking University,28.0,2.0
Shanghai Jiao Tong University,53.0,NaN
Tsinghua University,62.0,4.0


从上面可以看出这两个函数的区别在于，`crosstab`的对应位置传入的是具体的序列，而`pivot_table`传入的是被调用表对应的名字，若传入序列对应的值则会报错。


In [57]:
#除了默认状态下的count统计，所有的聚合字符串和返回标量的自定义函数都是可用的
#统计对应组合的身高均值
pd.crosstab(index = df.School, columns = df.Transfer, values = df.Height, aggfunc = 'mean')

Transfer,N,Y
School,,
Fudan University,162.043750,177.20
Peking University,163.429630,162.40
Shanghai Jiao Tong University,163.953846,NaN
Tsinghua University,163.253571,164.55


### 2. explode


In [58]:
df_ex = pd.DataFrame({'A': [[1, 2], 'my_str', {1, 2}, pd.Series([3, 4])],
                      'B': 1})
df_ex

,A,B
0,"[1, 2]",1
1,my_str,1
2,"{1, 2}",1
3,0 3 1 4 dtype: int64,1


In [59]:
#explode参数能够对某一列的元素进行纵向的展开，被展开的单元格必须存储list, tuple, Series, np.ndarray中的一种类型。
df_ex.explode('A')

,A,B
0,1,1
0,2,1
1,my_str,1
2,"{1, 2}",1
3,3,1
3,4,1


### 3. get_dummies


In [60]:
#get_dummies是用于特征构建的重要函数之一，其作用是把类别特征转为指示变量。
#对年级一列转为指示变量，属于某一个年级的对应列标记为1，否则为0
pd.get_dummies(df.Grade).head()

,Freshman,Junior,Senior,Sophomore
0,1,0,0,0
1,1,0,0,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1


## 四、练习
### Ex1：美国非法药物数据集

现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量：

In [62]:
df = pd.read_csv('./data/drugs.csv').sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4


1. 将数据转为如下的形式：

<img src="./source/_static/Ex5_1.png" width="35%">




In [64]:
df = pd.read_csv('data/drugs.csv').sort_values([
     'State','COUNTY','SubstanceName'],ignore_index=True)


res = df.pivot(index=['State','COUNTY','SubstanceName'
              ], columns='YYYY', values='DrugReports'
              ).reset_index().rename_axis(columns={'YYYY':''})


res.head(5)

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0


In [65]:
#2. 将第1问中的结果恢复为原表。
res_melted = res.melt(id_vars = ['State','COUNTY','SubstanceName'],
                     value_vars = res.columns[-8:],
                     var_name = 'YYYY',
                     value_name = 'DrugReports').dropna(
                     subset=['DrugReports'])


res_melted = res_melted[df.columns].sort_values([
              'State','COUNTY','SubstanceName'],ignore_index=True
              ).astype({'YYYY':'int64', 'DrugReports':'int64'})
res_melted.equals(df)

True

In [66]:
#3. 按`State`分别统计每年的报告数量总和，
#其中`State, YYYY`分别为列索引和行索引，要求分别使用`pivot_table`函数与`groupby+unstack`两种不同的策略实现，并体会它们之间的联系。
res = df.pivot_table(index='YYYY', columns='State',
                     values='DrugReports', aggfunc='sum')


res.head(3)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376


In [67]:
res = df.groupby(['State', 'YYYY'])['DrugReports'].sum(
                ).to_frame().unstack(0).droplevel(0,axis=1)


res.head(3)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376




### Ex2：特殊的wide_to_long方法

从功能上看，`melt`方法应当属于`wide_to_long`的一种特殊情况，即`stubnames`只有一类。请使用`wide_to_long`生成`melt`一节中的`df_melted`。（提示：对列名增加适当的前缀）

In [68]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [69]:
df = pd.DataFrame({'Class':[1,2],
                  'Name':['San Zhang', 'Si Li'],
                  'Chinese':[80, 90],
                  'Math':[80, 75]})


df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [70]:
df = df.rename(columns={'Chinese':'pre_Chinese', 'Math':'pre_Math'})

pd.wide_to_long(df,
                stubnames=['pre'],
                i = ['Class', 'Name'],
                j='Subject',
                sep='_',
                suffix='.+').reset_index().rename(columns={'pre':'Grade'})


,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75
